In [72]:
import pandas as pd
df=pd.read_csv("Commands Classification Dataset (version 1).csv")
df.drop(["Unnamed: 3","Cmd"],inplace=True,axis=1)

In [73]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report
import joblib

# Assuming df is your DataFrame
X = df['Commands']  # Input features
y = df['Class']      # Target variable

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Define the pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(max_features=10000)),
    ('classifier', MultinomialNB())
])

# Train the pipeline
pipeline.fit(X_train, y_train)

# Save the pipeline

joblib.dump(pipeline, 'joblib/ClassSvm.joblib')

# Predictions
y_pred = pipeline.predict(X_test)

# Evaluation
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

       Cloud       0.96      1.00      0.98        24
       Edge        1.00      0.95      0.97        55
      Update       0.89      0.96      0.93        26

    accuracy                           0.96       105
   macro avg       0.95      0.97      0.96       105
weighted avg       0.96      0.96      0.96       105



In [74]:
import pandas as pd
import joblib

# Load the saved pipeline
pipeline = joblib.load('joblib/ClassSvm.joblib')

# Create a DataFrame with a single row containing the command to classify
command_to_classify = "How long will it take to update"
data = {'Commands': [command_to_classify]}
single_command_df = pd.DataFrame(data)

# Predict the class
predicted_class = pipeline.predict(single_command_df['Commands'])

print("Predicted class:", predicted_class[0])


Predicted class: Update


In [75]:
df.columns

Index(['Class', 'Subclass', 'Commands'], dtype='object')

In [76]:
df["Class"].unique()

array(['Edge ', 'Cloud', 'Update'], dtype=object)

In [77]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import joblib

# Assuming df is your DataFrame
X = df['Commands']  # Input features
y = df['Subclass']   # Target variable

# Iterate over each unique class
unique_classes = df["Class"].unique()

# Dictionary to store models
models = {}

for class_label in unique_classes:
    print("Training model for class:", class_label)
    
    # Filter the DataFrame for the specific class
    class_df = df[df["Class"] == class_label]
    X_class = class_df['Commands']
    y_class = class_df['Subclass']
    
    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X_class, y_class, test_size=0.2, random_state=42)
    
    # Define the pipeline for the current class
    pipeline = Pipeline([
        ('tfidf', TfidfVectorizer(max_features=10000)),
        ('classifier', MultinomialNB())
    ])
    
    # Train the pipeline
    pipeline.fit(X_train, y_train)
    
    # Store the model in the dictionary
    models[class_label] = pipeline

    # Save the model
    joblib.dump(pipeline, f'joblib/SubClassmodel_{class_label}.joblib')

    # Evaluate the model
    y_pred = pipeline.predict(X_test)
    print("Classification report for class:", class_label)
    print(classification_report(y_test, y_pred))

# Now you have models for each class stored in the 'models' dictionary


Training model for class: Edge 
Classification report for class: Edge 
                  precision    recall  f1-score   support

         Battery       0.63      0.92      0.75        13
            Fuel       0.64      0.82      0.72        11
       Greetings       1.00      0.20      0.33         5
Safety reminders       1.00      0.14      0.25         7
           Tires       0.85      0.92      0.88        12

        accuracy                           0.71        48
       macro avg       0.82      0.60      0.59        48
    weighted avg       0.78      0.71      0.66        48

Training model for class: Cloud
Classification report for class: Cloud
               precision    recall  f1-score   support

         News       1.00      1.00      1.00         7
Notifications       1.00      0.71      0.83         7
  Songs/media       1.00      1.00      1.00         3
      Traffic       1.00      1.00      1.00         5
      Weather       0.80      1.00      0.89         8

 

In [78]:
# Load the saved model for a specific class
class_label_to_predict = predicted_class[0]
model = joblib.load(f'joblib/SubClassmodel_{class_label}.joblib')

# Create a DataFrame with a single row containing the command to classify
command_to_classify = "How long will it take to update"
data = {'Commands': [command_to_classify]}
single_command_df = pd.DataFrame(data)

# Predict the subclass for the given command and class
predicted_subclass = model.predict(single_command_df['Commands'])

print("Predicted subclass for class", class_label_to_predict, ":", predicted_subclass[0])


Predicted subclass for class Update : Check
